TRANSFER LEARNING: Esto es lo que sí vamos a usar 

In [2]:
import os #El sistema
from torch import optim, nn, utils, Tensor #Para crear las redes neuronales, los optimizadores, manejar los datos y hacer operaciones complejas con tensores 
from torchvision.datasets import MNIST #Este es un conjunto de datos de ejemplo
from torchvision.transforms import ToTensor #Esto nos va a ayudar a transformar 
import lightning as L #Lightning ahora sí 
import torch.nn.functional as F

In [4]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3)) #Compresión de la imagen

    def forward(self, x): #Lo que hay que hacer con las entradas
        return self.l1(x)


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28)) #Descompresión

    def forward(self, x):
        return self.l1(x)

In [5]:
class LitAutoEncoder(L.LightningModule): #Este deberia ser un modulo completo para realizar el entrenamiento
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder #Este modulo lo que estaría haciendo es unir el compresor y el descompresor que configuramos anteriormente
        self.decoder = decoder #Aquí hacemos entonces, la conexión con los nn Modules

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        return loss #Retorna la perdida 
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 

In [16]:
class CIFAR10Classifier(L.LightningModule):
    def __init__(self):
        super().__init__()
        # init the pretrained LightningModule
        self.feature_extractor = AutoEncoder.load_from_checkpoint(PATH).encoder
        self.feature_extractor.freeze()

        # the autoencoder outputs a 100-dim representation and CIFAR-10 has 10 classes
        self.classifier = nn.Linear(100, 10)

    def forward(self, x):
        representations = self.feature_extractor(x)
        x = self.classifier(representations)
        #Qué hace falta?
        return x
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1) #Porqué es necesario aplastar la imagen???
        z = self.encoder(x) #Codificamos 
        x_hat = self.decoder(z) #Decodificamos
        loss = F.mse_loss(x_hat, x) #Calcula función de perdida que se tuvo en la reconstrucción
        return loss #Retorna la perdida 
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 
    

EJEMPLOO
_____________________________________________-


In [26]:
import torchvision.models as models


class ImagenetTransferLearning(L.LightningModule): 
    def __init__(self, num_target_classes=10, lr=1e-3):
        super().__init__()

        # init a pretrained resnet
        backbone = models.resnet50(weights="DEFAULT") #Esta es la red que ya existía
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)
        self.feature_extractor.eval()

        # use the pretrained model to classify cifar-10 (10 image classes)
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes) 
        self.lr = lr 
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        return x

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x_hat = self(x)
        loss = self.loss_fn(x_hat, _) # CrossEntropyLoss para clasificación 
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True) 
        return loss
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3) #Aquí el optimizador ajusta los pesos del modelo 
        return optimizer #Regresa el optimizador 
        ...

In [20]:
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor()) #Descargamos la base de datos, las imagenes y las transformamos en tensores
train_loader = utils.data.DataLoader(dataset) # Esto clasifica las imagenes de la base en batches 

In [27]:
import torch
model = ImagenetTransferLearning()
trainer = L.Trainer()
trainer.fit(model, train_dataloaders=train_loader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name              | Type             | Params | Mode 
---------------------------------------------------------------
0 | feature_extractor | Sequential       | 23.5 M | eval 
1 | classifier        | Linear           | 20.5 K | train
2 | loss_fn           | CrossEntropyLoss | 0      | train
---------------------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.114    Total estimated model params size (MB)
2         Modules in train mode
150       Modules in eval mode


Training: |                                                                                      | 0/? [00:00<…

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 1, 28, 28] to have 3 channels, but got 1 channels instead